# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [4]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
#os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4o", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-07-10 17:21:39,508 - 131550697652224 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
To tackle the task of continuously monitoring and analyzing market data for AAPL, we will need to involve multiple co-workers, each with their specialized skills. Here are the steps I plan to take:

1. **Delegate the task of gathering and analyzing current market data for AAPL to the Data Analyst.**
2. **Ask the Trading Strategy Developer to use statistical modeling and machine learning to identify trends and predict market movements based on the data.**
3. **Ask the Trade Advisor for insights on significant market opportunities or threats for AAPL based on the trends and predictions.**
4. **Ask the Risk Advisor to assess potential risks associated with the identified trends and predictions.**

Let's start by dele

In [18]:
from IPython.display import Markdown
Markdown(result)

### Comprehensive Risk Analysis Report for AAPL Trading Strategies

#### 1. Momentum Trading
- **Risks:**
  - Market volatility, false signals from technical indicators.
- **Safeguards:**
  - Position sizing, diversification, regular review, and tight stop-loss orders.
- **Mitigation Strategies:**
  - **Refine Entry Criteria:** Use tighter entry criteria such as combining multiple indicators (e.g., RSI with MACD and moving averages) to confirm momentum.
  - **Dynamic Stop-Losses:** Implement dynamic stop-loss orders that adjust based on volatility, reducing the risk of premature exits.
  - **Stress Testing:** Conduct stress tests on historical data to understand how momentum trading strategies perform in various market conditions.

#### 2. Breakout Trading
- **Risks:**
  - False breakouts, reliance on volume confirmation.
- **Safeguards:**
  - Trailing stops, favorable risk-reward ratio, volume confirmation.
- **Mitigation Strategies:**
  - **False Breakout Filters:** Utilize additional indicators such as the Average True Range (ATR) to filter out false breakouts.
  - **Partial Exits:** Implement partial exit strategies to lock in profits as the price progresses towards the target, reducing exposure.
  - **Volume Confirmation:** Ensure substantial volume confirmation before entering trades to validate breakouts.

#### 3. Scalping
- **Risks:**
  - Quick market movements, high trading costs, leverage risks.
- **Safeguards:**
  - Quick exits, high liquidity, time management, minimal leverage.
- **Mitigation Strategies:**
  - **Enhanced Execution Tools:** Use advanced order types and algorithms to ensure optimal execution and minimize slippage.
  - **Real-time Analytics:** Employ real-time analytics to monitor market depth and order flow, aiding in better decision-making.
  - **Risk Limits:** Set strict daily risk limits to cap potential losses from multiple small trades.

#### 4. Pairs Trading
- **Risks:**
  - Correlation risks, market neutrality challenges.
- **Safeguards:**
  - Regular correlation analysis, diversification, regular rebalancing.
- **Mitigation Strategies:**
  - **Dynamic Hedging:** Implement dynamic hedging strategies to adjust positions based on real-time correlation changes.
  - **Statistical Arbitrage Models:** Use advanced statistical models to identify mispricings and improve the accuracy of entry and exit points.
  - **Regular Audits:** Conduct regular audits of the pairs trading strategy to ensure it remains market-neutral and effective.

#### 5. News-Based Trading
- **Risks:**
  - Volatility, information accuracy, quick reaction errors.
- **Safeguards:**
  - Event analysis, stop-loss orders, information verification.
- **Mitigation Strategies:**
  - **Sentiment Analysis Tools:** Integrate sentiment analysis tools to gauge market reactions to news in real-time.
  - **Automated Alerts:** Set up automated alerts for significant news events to ensure timely execution.
  - **Scenario Planning:** Develop multiple scenarios for potential news outcomes to prepare for various market reactions.

#### 6. Algorithmic Trading
- **Risks:**
  - Algorithm failure, lack of human oversight, backtesting limitations.
- **Safeguards:**
  - Backtesting, risk parameters, real-time monitoring, fail-safes, diversification.
- **Mitigation Strategies:**
  - **Robust Backtesting:** Perform extensive backtesting under different market conditions and stress scenarios to ensure algorithm robustness.
  - **Risk-Adjusted Metrics:** Use risk-adjusted performance metrics (e.g., Sharpe ratio) to evaluate algorithm effectiveness.
  - **Continuous Learning:** Implement machine learning techniques to allow the algorithm to adapt and improve based on new data.

### Conclusion
Each trading strategy for AAPL carries its own set of risks and requires specific safeguards and mitigation strategies to ensure alignment with the firm’s risk tolerance. By implementing these robust risk management practices and mitigation strategies, traders can effectively manage potential losses and maximize trading success.